<a href="https://colab.research.google.com/github/Smpests/KeepLearning/blob/master/jupyter-notebook/Python%E5%85%83%E7%B1%BB%E5%92%8C%E8%A3%85%E9%A5%B0%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
# 元类实现单例模式

""" 
写在前面：
元类（metaclass）的作用类似于类的类或者实例工厂，
区别于基类，它不做抽象，只通过重写与实例化、初始化的相关方法，来改变对象的实例化过程和表现。
基类则是在顶层抽象，对类进行扩展。
"""
class TestMeta(type):
  def __init__(cls, *args, **kwargs):
    cls.__instance = None
    super().__init__(*args, **kwargs)
    print("this is meta")

  def __new__(cls, *args, **kwargs):
    print("this is meta __new__")
    return super().__new__(cls, *args, **kwargs)
  
  def __call__(cls, *args, **kwargs):
    if cls.__instance is None:
      print("create")
      cls.__instance = super().__call__(*args, **kwargs)
    return cls.__instance

class TestBase(object):
  def __init__(self):
    super().__init__()
    print("this is father")
  
  def __new__(cls, weight=20, *args, **kwargs):
    print("this is base __new__")
    instance = super().__new__(cls, *args, **kwargs)
    instance.weight = weight
    return instance
  def __call__(cls, *args, **kwargs):
    return super().__call__(*args, **kwargs)


class Test(TestBase, metaclass=TestMeta):
  def __init__(self, name, age):
    super().__init__()
    self.name = name
    self.age = age
    print("this is son")

  def __new__(cls, *args, **kwargs):
    instance = super().__new__(cls)
    print("this is son __new__")
    return instance

class T2(metaclass=TestMeta):

  def __new__(cls, *args, **kwargs):
    return super().__new__(cls)


t = Test("Lily", 18)
print(t.__dict__)
u = Test("Xiaoming", 20)
print(u.__dict__)
t2 = T2()


this is meta __new__
this is meta
this is meta __new__
this is meta
create
this is base __new__
this is son __new__
this is father
this is son
{'weight': 20, 'name': 'Lily', 'age': 18}
{'weight': 20, 'name': 'Lily', 'age': 18}
create


In [61]:
from functools import wraps

# 通过装饰器实现单例对象，维护一个单例对象哈希表

# 装饰器函数
def singleton(cls):
  instances = {}
  
  # 不写@wraps(cls)无法获取cls类的原始属性，如name、doc等基础信息，不写得到的是装饰器方法的信息
  @wraps(cls)
  def wrapper(*args, **kwargs):
    if cls not in instances:
      instances[cls] = cls(*args, **kwargs)
    return instances[cls]
  return wrapper

# 装饰器类
class Decorator:
  instances = {}
  def __init__(self, cls):
    self.cls = cls
  
  def __call__(self, *args, **kwargs):
    if self.cls not in self.instances:
      self.instances[self.cls] = self.cls(*args, **kwargs)     
    return self.instances[self.cls]

@singleton
class SingletonClass:
  def __init__(self, name):
    self.name = name

@Decorator
class SingletonClass2:
  def __init__(self, name):
    self.name = name

s1 = SingletonClass("小张")
s2 = SingletonClass("小李")
print(s1.__dict__)
print(s2.__dict__)

s3 = SingletonClass2("小王")
s4 = SingletonClass2("小刘")
print(s3.__dict__)
print(s4.__dict__)

{'name': '小张'}
{'name': '小张'}
{'name': '小王'}
{'name': '小王'}


In [69]:
from copy import copy, deepcopy
from functools import wraps

# 基于元类的原型模式
class PrototypeMeta(type):
  def __init__(cls, *args, **kwargs):
    super().__init__(*args, **kwargs)
    cls.clone = lambda self, deep=True: deepcopy(self) if deep else copy(self)

class CloneableClass(metaclass=PrototypeMeta):
  def __init__(self, names=[]):
    self.names = names

a = CloneableClass(["老马", "茄子"])
b = a.clone()
c = a.clone(deep=False)
b.names.append("老朴")
c.names.append("枣子")
print(a.__dict__)
print(b.__dict__)
print(c.__dict__)

{'names': ['老马', '茄子', '枣子']}
{'names': ['老马', '茄子', '老朴']}
{'names': ['老马', '茄子', '枣子']}
